### Installation of Data

In [1]:
!nvidia-smi -L

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
! pip install -q kaggle

In [3]:
#upload kaggle.json file
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sujalthakkar","key":"f1c0c5709adf673907dc35bf54027e00"}'}

In [4]:
! mkdir ~/.kaggle

In [5]:
! cp kaggle.json ~/.kaggle/

In [6]:
#Permission
! chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle datasets download -d arashnic/book-recommendation-dataset

Dataset URL: https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset
License(s): CC0-1.0
 49% 12.0M/24.3M [00:00<00:00, 125MB/s]
100% 24.3M/24.3M [00:00<00:00, 181MB/s]


In [8]:
!unzip book-recommendation-dataset.zip

Archive:  book-recommendation-dataset.zip
  inflating: Books.csv               
  inflating: DeepRec.png             
  inflating: Ratings.csv             
  inflating: Users.csv               
  inflating: classicRec.png          
  inflating: recsys_taxonomy2.png    


### Diffetrent types of Recommendation systems:

1) **Content-based recommenders:** These recommenders suggest items similar to the ones a user has already liked or consumed. They analyze the attributes of the items that the user has already rated or interacted with and recommend other items with similar attributes.

2) **Collaborative filtering recommenders:** These recommenders rely on the behavior of a large number of users to make recommendations. They look at the preferences of similar users and recommend items that are popular among those users.

3) **Hybrid recommenders:** These recommenders combine the approaches of content-based and collaborative filtering recommenders to generate more accurate and diverse recommendations.

4) **Knowledge-based recommenders:** These recommenders use a knowledge base to recommend items to users. They use information about the user's preferences and needs, as well as information about the items themselves, to make recommendations.

5) **Demographic-based recommenders:** These recommenders make recommendations based on demographic information such as age, gender, income, and education. They use this information to predict what products or services a user may be interested in.

6) **Context-based recommenders:** These recommenders make recommendations based on the context in which the user is operating, such as time of day, location, or weather. They use this information to suggest relevant items or services.

### Loading the dependencies

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import sklearn
from sklearn.model_selection import train_test_split

In [10]:
books =pd.read_csv("/content/Books.csv")
Ratings =pd.read_csv("/content/Ratings.csv")
Users =pd.read_csv("/content/Users.csv")

<ipython-input-10-e0d5ded4bcc4>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books =pd.read_csv("/content/Books.csv")


In [11]:
books.head(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...


In [12]:
books.isna().sum()

ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [13]:
books.columns,len(books.columns)

(Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
        'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
       dtype='object'),
 8)

In [14]:
Ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [15]:
Ratings.isna().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [16]:
Users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [17]:
Users.isna().sum() #we are not going to consider the age col of the USER so not to worry

User-ID          0
Location         0
Age         110762
dtype: int64

In [18]:
print(books.shape)
print(Users.shape)
print(Ratings.shape)

(271360, 8)
(278858, 3)
(1149780, 3)


In [19]:
books.duplicated().sum()

0

In [20]:
Users.duplicated().sum()

0

In [21]:
Ratings.duplicated().sum()

0

### Popularity Based Recommendation System

**NOTE:** Only we will be consider those book who has got minimum of 250 votes.

In [22]:
books

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [23]:
len(books),books.columns

(271360,
 Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
        'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
       dtype='object'))

In [24]:
Ratings.columns

Index(['User-ID', 'ISBN', 'Book-Rating'], dtype='object')

In [25]:
## since both book & rating both are having ISBN column we will be merging the dataset on ISBN
Ratings_with_name=Ratings.merge(books,on="ISBN")

In [26]:
Ratings_with_name.columns, Ratings_with_name.shape

(Index(['User-ID', 'ISBN', 'Book-Rating', 'Book-Title', 'Book-Author',
        'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M',
        'Image-URL-L'],
       dtype='object'),
 (1031136, 10))

In [27]:
Ratings_with_name

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
...,...,...,...,...,...,...,...,...,...,...
1031131,276688,0517145553,0,Mostly Harmless,Douglas Adams,1995,Random House Value Pub,http://images.amazon.com/images/P/0517145553.0...,http://images.amazon.com/images/P/0517145553.0...,http://images.amazon.com/images/P/0517145553.0...
1031132,276688,1575660792,7,Gray Matter,Shirley Kennett,1996,Kensington Publishing Corporation,http://images.amazon.com/images/P/1575660792.0...,http://images.amazon.com/images/P/1575660792.0...,http://images.amazon.com/images/P/1575660792.0...
1031133,276690,0590907301,0,Triplet Trouble and the Class Trip (Triplet Tr...,Debbie Dadey,1997,Apple,http://images.amazon.com/images/P/0590907301.0...,http://images.amazon.com/images/P/0590907301.0...,http://images.amazon.com/images/P/0590907301.0...
1031134,276704,0679752714,0,A Desert of Pure Feeling (Vintage Contemporaries),Judith Freeman,1997,Vintage Books USA,http://images.amazon.com/images/P/0679752714.0...,http://images.amazon.com/images/P/0679752714.0...,http://images.amazon.com/images/P/0679752714.0...


In [28]:
#since there were nearly 11lac data inthe rating csv file but some of them were not matching withthe book dataset so we
## finally got less data...nearly 10lac

In [29]:
Ratings_with_name.columns

Index(['User-ID', 'ISBN', 'Book-Rating', 'Book-Title', 'Book-Author',
       'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M',
       'Image-URL-L'],
      dtype='object')

In [30]:
#only consider those books who is having a vote of 250+ on them
num_rating_df=Ratings_with_name.groupby("Book-Title").count()["Book-Rating"].reset_index()
num_rating_df.rename(columns={"Book-Rating":"num_ratings"}, inplace=True)
num_rating_df.head()

,Book-Title,num_ratings
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1


In [31]:
num_rating_df["num_ratings"].value_counts()

num_ratings
1      127513
2       42729
3       20255
4       11364
5        7529
        ...  
588         1
575         1
428         1
347         1
234         1
Name: count, Length: 377, dtype: int64

In [32]:
Users.head(2)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0


In [33]:
Users.columns

Index(['User-ID', 'Location', 'Age'], dtype='object')

In [34]:
#only consider those books who is having a vote of 250+ on them
avg_rating_df=Ratings_with_name.groupby("Book-Title")["Book-Rating"].mean().reset_index()
avg_rating_df.rename(columns={"Book-Rating":"avg_ratings"}, inplace=True)
avg_rating_df.head()

,Book-Title,avg_ratings
0,A Light in the Storm: The Civil War Diary of ...,2.25
1,Always Have Popsicles,0.00
2,Apple Magic (The Collector's series),0.00
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.00
4,Beyond IBM: Leadership Marketing and Finance ...,0.00


In [35]:
popular_df = num_rating_df.merge(avg_rating_df,on="Book-Title")
popular_df.head()

,Book-Title,num_ratings,avg_ratings
0,A Light in the Storm: The Civil War Diary of ...,4,2.25
1,Always Have Popsicles,1,0.00
2,Apple Magic (The Collector's series),1,0.00
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1,8.00
4,Beyond IBM: Leadership Marketing and Finance ...,1,0.00


In [36]:
popular_df

,Book-Title,num_ratings,avg_ratings
0,A Light in the Storm: The Civil War Diary of ...,4,2.250000
1,Always Have Popsicles,1,0.000000
2,Apple Magic (The Collector's series),1,0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1,8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,1,0.000000
...,...,...,...
241066,Ã?Â?lpiraten.,2,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,4,5.250000
241068,Ã?Â?sterlich leben.,1,7.000000
241069,Ã?Â?stlich der Berge.,3,2.666667


In [37]:
popular_df=popular_df[popular_df["num_ratings"]>=250].sort_values("avg_ratings",ascending=False).head(50)

In [38]:
df=popular_df.merge(books, on="Book-Title").drop_duplicates("Book-Title")

In [39]:
df.columns

Index(['Book-Title', 'num_ratings', 'avg_ratings', 'ISBN', 'Book-Author',
       'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M',
       'Image-URL-L'],
      dtype='object')

In [40]:
df=df[["Book-Title","Book-Author","Image-URL-M","num_ratings","avg_ratings"]]

In [41]:
df.shape

(50, 5)

In [42]:
df

,Book-Title,Book-Author,Image-URL-M,num_ratings,avg_ratings
0,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,http://images.amazon.com/images/P/0439136350.0...,428,5.852804
3,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,http://images.amazon.com/images/P/0439139597.0...,387,5.824289
5,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,http://images.amazon.com/images/P/0590353403.0...,278,5.737410
9,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,http://images.amazon.com/images/P/043935806X.0...,347,5.501441
13,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,http://images.amazon.com/images/P/0439064872.0...,556,5.183453
16,The Hobbit : The Enchanting Prelude to The Lor...,J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339681.0...,281,5.007117
17,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339703.0...,368,4.948370
26,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,http://images.amazon.com/images/P/059035342X.0...,575,4.895652
28,"The Two Towers (The Lord of the Rings, Part 2)",J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339711.0...,260,4.880769
39,To Kill a Mockingbird,Harper Lee,http://images.amazon.com/images/P/0446310786.0...,510,4.700000


### colleborative Recommendation system

In [43]:
Ratings.columns

Index(['User-ID', 'ISBN', 'Book-Rating'], dtype='object')

In [44]:
Ratings_with_name.columns

Index(['User-ID', 'ISBN', 'Book-Rating', 'Book-Title', 'Book-Author',
       'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M',
       'Image-URL-L'],
      dtype='object')

In [45]:
Ratings_with_name

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
...,...,...,...,...,...,...,...,...,...,...
1031131,276688,0517145553,0,Mostly Harmless,Douglas Adams,1995,Random House Value Pub,http://images.amazon.com/images/P/0517145553.0...,http://images.amazon.com/images/P/0517145553.0...,http://images.amazon.com/images/P/0517145553.0...
1031132,276688,1575660792,7,Gray Matter,Shirley Kennett,1996,Kensington Publishing Corporation,http://images.amazon.com/images/P/1575660792.0...,http://images.amazon.com/images/P/1575660792.0...,http://images.amazon.com/images/P/1575660792.0...
1031133,276690,0590907301,0,Triplet Trouble and the Class Trip (Triplet Tr...,Debbie Dadey,1997,Apple,http://images.amazon.com/images/P/0590907301.0...,http://images.amazon.com/images/P/0590907301.0...,http://images.amazon.com/images/P/0590907301.0...
1031134,276704,0679752714,0,A Desert of Pure Feeling (Vintage Contemporaries),Judith Freeman,1997,Vintage Books USA,http://images.amazon.com/images/P/0679752714.0...,http://images.amazon.com/images/P/0679752714.0...,http://images.amazon.com/images/P/0679752714.0...


In [46]:
Ratings_with_name.groupby("User-ID").count()["Book-Rating"] #out of all we do have only 92k users who has rated the book

User-ID
2          1
8         17
9          3
10         1
12         1
          ..
278846     1
278849     4
278851    23
278852     1
278854     8
Name: Book-Rating, Length: 92106, dtype: int64

In [47]:
Ratings_with_name.groupby("User-ID").count()["Book-Rating"]>100

User-ID
2         False
8         False
9         False
10        False
12        False
          ...  
278846    False
278849    False
278851    False
278852    False
278854    False
Name: Book-Rating, Length: 92106, dtype: bool

In [48]:
x=Ratings_with_name.groupby("User-ID").count()["Book-Rating"]>100
value_users=x[x].index
value_users

Index([   254,    507,    882,   1424,   1435,   1733,   1903,   2033,   2110,
         2276,
       ...
       275020, 275970, 276463, 276680, 277427, 277478, 277639, 278137, 278188,
       278418],
      dtype='int64', name='User-ID', length=1648)

Uptill now we have fetched the user_id of the users who has actually votes for the book whose value of rating is more then 100 votes/ratings....

In [49]:
filtered_rating= Ratings_with_name[Ratings_with_name["User-ID"].isin(value_users)]

In [50]:
filtered_rating.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
5,23768,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
7,28523,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
14,77480,034545104X,8,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...


In [51]:
filtered_rating.columns,len(filtered_rating.columns)

(Index(['User-ID', 'ISBN', 'Book-Rating', 'Book-Title', 'Book-Author',
        'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M',
        'Image-URL-L'],
       dtype='object'),
 10)

In [52]:
y=filtered_rating.groupby("Book-Title").count()["Book-Rating"] >=50

In [53]:
famous_books = y[y].index

In [54]:
final_ratings=filtered_rating[filtered_rating["Book-Title"].isin(famous_books)]

In [55]:
final_ratings.drop_duplicates()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
63,278418,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...
65,3363,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...
66,7158,0446520802,10,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...
67,8253,0446520802,10,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...
69,11676,0446520802,10,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...
...,...,...,...,...,...,...,...,...,...,...
1028777,271284,0440910927,0,The Rainmaker,John Grisham,1995,Island,http://images.amazon.com/images/P/0440910927.0...,http://images.amazon.com/images/P/0440910927.0...,http://images.amazon.com/images/P/0440910927.0...
1029070,271705,B0001PIOX4,0,Fahrenheit 451,Ray Bradbury,1993,Simon &amp; Schuster,http://images.amazon.com/images/P/B0001PIOX4.0...,http://images.amazon.com/images/P/B0001PIOX4.0...,http://images.amazon.com/images/P/B0001PIOX4.0...
1029756,274004,037572799X,0,In Pursuit of the Proper Sinner,Elizabeth George,2000,Random House Large Print Publishing,http://images.amazon.com/images/P/037572799X.0...,http://images.amazon.com/images/P/037572799X.0...,http://images.amazon.com/images/P/037572799X.0...
1030363,274808,0449701913,0,Homecoming,Cynthia Voigt,1983,Fawcett Books,http://images.amazon.com/images/P/0449701913.0...,http://images.amazon.com/images/P/0449701913.0...,http://images.amazon.com/images/P/0449701913.0...


In [56]:
final_ratings.columns

Index(['User-ID', 'ISBN', 'Book-Rating', 'Book-Title', 'Book-Author',
       'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M',
       'Image-URL-L'],
      dtype='object')

In [57]:
pt=final_ratings.pivot_table(index="Book-Title",columns="User-ID",values="Book-Rating")

In [58]:
pt.shape

(1070, 1634)

In [59]:
pt.fillna(0,inplace=True)

In [60]:
pt

User-ID,254,507,882,1424,1435,1733,1903,2033,2110,2276,...,275020,275970,276463,276680,277427,277478,277639,278137,278188,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010: Odyssey Two,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
204 Rosewood Lane,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24 Hours,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**every book is in dimension of 1634 vector...there is 1 book who has 1634 values which is nothing but the vote which users have had given to the book**

NOTE: and now we are going to calculate the euclidean distance betweeen every vector of the book so we can know which are the books who has less Euclidean distance those books are similar and can be recommended when in need

USING: **COSINE_SIMILARITy**

In [61]:
from sklearn.metrics.pairwise import cosine_similarity

In [62]:
cosine_similarity(pt).shape #we are getting a 2D vector
#this means that we have successfully calculated 1070 books euclidean distance with 1070 books

(1070, 1070)

In [63]:
similarity_score=cosine_similarity(pt)
similarity_score[0]

array([1.        , 0.06550754, 0.09317747, ..., 0.07547772, 0.05810769,
       0.03029366])

In [64]:
len(similarity_score[0]) #the above we can see the similarity score of one book with another.......

1070

In [65]:
index=np.where(pt.index=="Zoya")[0][0]
index

1068

In [66]:
list(enumerate(similarity_score[0])) #we got the list of all the books similarity score with respect to "0" book title

[(0, 0.9999999999999999),
 (1, 0.0655075384044448),
 (2, 0.09317746621659334),
 (3, 0.0),
 (4, 0.009722316399413395),
 (5, 0.008054094650987997),
 (6, 0.0),
 (7, 0.070551332689024),
 (8, 0.03423204512344849),
 (9, 0.0),
 (10, 0.03379284353649112),
 (11, 0.06681636361132519),
 (12, 0.047626939641346885),
 (13, 0.06441592815945904),
 (14, 0.03524833057168071),
 (15, 0.0),
 (16, 0.0),
 (17, 0.009204071581260418),
 (18, 0.023964214596610765),
 (19, 0.024470201632145677),
 (20, 0.04893499518611978),
 (21, 0.017394573947202147),
 (22, 0.04266751800588524),
 (23, 0.013202225310401534),
 (24, 0.007560985471428589),
 (25, 0.05478340464153213),
 (26, 0.04900058247649345),
 (27, 0.09291532530701768),
 (28, 0.050415390187525035),
 (29, 0.07653479411446046),
 (30, 0.03450778206808572),
 (31, 0.0170106735352967),
 (32, 0.08986696436517477),
 (33, 0.0),
 (34, 0.0),
 (35, 0.0),
 (36, 0.0895372509539337),
 (37, 0.049547580524288586),
 (38, 0.04177355944189604),
 (39, 0.0801951868658444),
 (40, 0.057634

In [67]:
sorted(list(enumerate(similarity_score[0])),key=lambda x:x[1],reverse=True)[1:11]

[(75, 0.23331416605380295),
 (127, 0.2080470853928461),
 (663, 0.19915207689336847),
 (88, 0.18387183815857203),
 (827, 0.18088630162996125),
 (600, 0.17967431783991286),
 (754, 0.1782421419783686),
 (440, 0.17809974783929494),
 (963, 0.17314602656709555),
 (989, 0.17173944042904682)]

In [68]:
"""
#make a function recommendation
#which will take input as a book-title/name and it will provide me a list of 5to10 books who are similar to the given book name
def recommend(book_name):
  #fetch the index / on which number/index the book is in the dataset
  index=np.where(pt.index==book_name)[0][0]
  similar_items=sorted(list(enumerate(similarity_score[index])),key=lambda x:x[1],reverse=True)[1:11]
  for i in similar_items:
    print(pt.index[i[0]]) #this will give us all the index values of the other books
"""

'\n#make a function recommendation\n#which will take input as a book-title/name and it will provide me a list of 5to10 books who are similar to the given book name\ndef recommend(book_name):\n  #fetch the index / on which number/index the book is in the dataset\n  index=np.where(pt.index==book_name)[0][0]\n  similar_items=sorted(list(enumerate(similarity_score[index])),key=lambda x:x[1],reverse=True)[1:11]\n  for i in similar_items:\n    print(pt.index[i[0]]) #this will give us all the index values of the other books\n'

In [69]:
def recommend(book_name):
    # Convert book_name to lowercase for case-insensitive matching
    book_name_lower = book_name.lower()

    # Find similar books based on partial matching
    similar_books = []
    for title in pt.index:
        if book_name_lower in title.lower():
            similar_books.append(title)


    # If no exact match is found, suggest similar books
    if len(similar_books) == 0:
        print("There is no book with that exact name.")
        print("Here are some books you might be looking for:")
        similar_items = sorted(list(enumerate(similarity_score[index])), key=lambda x: x[1], reverse=True)[1:11]
        recommended_book = pt.index[similar_items[0][0]] if similar_items else None
        print("Recommended book:", recommended_book)
    else:
        # Find index of recommended_book
        index = np.where(pt.index == similar_books[0])[0][0]

        # Find similar books based on cosine similarity score
        similar_items = sorted(list(enumerate(similarity_score[index])), key=lambda x: x[1], reverse=True)[1:11]
        recommendation_list = [pt.index[i[0]] for i in similar_items]
        print(" ")
        print("Recommendation list based on :", similar_books[0] + ":")
        print(" ")
        for book in recommendation_list:
            print(book)


In [70]:
recommend("19")

 
Recommendation list based on : 1984:
 
Animal Farm
Brave New World
Slaughterhouse Five or the Children's Crusade: A Duty Dance With Death
Awakening
The Handmaid's Tale
Roll of Thunder, Hear My Cry
The Catcher in the Rye
Lord of the Flies
The Vampire Lestat (Vampire Chronicles, Book II)
Timeline


In [71]:
recommend("1984")

 
Recommendation list based on : 1984:
 
Animal Farm
Brave New World
Slaughterhouse Five or the Children's Crusade: A Duty Dance With Death
Awakening
The Handmaid's Tale
Roll of Thunder, Hear My Cry
The Catcher in the Rye
Lord of the Flies
The Vampire Lestat (Vampire Chronicles, Book II)
Timeline


In [72]:
#recommend("The Last Song")
#pt.columns
"Sanctuary" in final_ratings["Book-Title"].values

True

In [73]:
recommend("Sanctuary")

 
Recommendation list based on : Sanctuary:
 
River's End
Homeport
Genuine Lies
Sea Swept (Quinn Brothers (Paperback))
Carnal Innocence
Midnight Bayou
Jewels of the Sun (Irish Trilogy)
The Reef
Brazen Virtue
Face the Fire (Three Sisters Island Trilogy)


In [74]:
recommend("A Walk to Remember")

 
Recommendation list based on : A Walk to Remember:
 
The Rescue
Nights in Rodanthe
The Notebook
A Bend in the Road
Message in a Bottle
Outbreak
Granny Dan
Chicken Soup for the Teenage Soul (Chicken Soup for the Soul)
The Secret Garden
That Camden Summer


In [75]:
recommend("The Da Vinci Code")

 
Recommendation list based on : The Da Vinci Code:
 
Angels &amp; Demons
Middlesex: A Novel
The Blue Nowhere : A Novel
Deception Point
Digital Fortress : A Thriller
The Lovely Bones: A Novel
Touching Evil
TickTock
Mystic River
The Sweet Potato Queens' Book of Love


In [76]:
recommend("The Hobbit : The Enchanting Prelude to The Lord of the Rings")

 
Recommendation list based on : The Hobbit : The Enchanting Prelude to The Lord of the Rings:
 
The Two Towers (The Lord of the Rings, Part 2)
The Fellowship of the Ring (The Lord of the Rings, Part 1)
The Return of the King (The Lord of the Rings, Part 3)
Lord of the Flies
The English Patient
Where the Red Fern Grows
Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))
Harry Potter and the Prisoner of Azkaban (Book 3)
The Catcher in the Rye
One for the Money (A Stephanie Plum Novel)


In [77]:
recommend("Harry Potter and the Prisoner of Azkaban (Book 3)")

 
Recommendation list based on : Harry Potter and the Prisoner of Azkaban (Book 3):
 
Harry Potter and the Goblet of Fire (Book 4)
Harry Potter and the Chamber of Secrets (Book 2)
Harry Potter and the Order of the Phoenix (Book 5)
Harry Potter and the Sorcerer's Stone (Book 1)
Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))
The Fellowship of the Ring (The Lord of the Rings, Part 1)
Charlie and the Chocolate Factory
Charlotte's Web (Trophy Newbery)
Dune Messiah (Dune Chronicles, Book 2)
The Two Towers (The Lord of the Rings, Part 2)


In [78]:
recommend("Harry Potter and the Goblet of Fire (Book 4)")

 
Recommendation list based on : Harry Potter and the Goblet of Fire (Book 4):
 
Harry Potter and the Prisoner of Azkaban (Book 3)
Harry Potter and the Chamber of Secrets (Book 2)
Harry Potter and the Order of the Phoenix (Book 5)
Harry Potter and the Sorcerer's Stone (Book 1)
Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))
The Fellowship of the Ring (The Lord of the Rings, Part 1)
Bridget Jones's Diary
The Bad Beginning (A Series of Unfortunate Events, Book 1)
The Two Towers (The Lord of the Rings, Part 2)
A Walk to Remember


In [79]:
recommend("harry potter")

 
Recommendation list based on : Harry Potter and the Chamber of Secrets (Book 2):
 
Harry Potter and the Prisoner of Azkaban (Book 3)
Harry Potter and the Goblet of Fire (Book 4)
Harry Potter and the Sorcerer's Stone (Book 1)
Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))
Harry Potter and the Order of the Phoenix (Book 5)
The Fellowship of the Ring (The Lord of the Rings, Part 1)
Charlotte's Web (Trophy Newbery)
Bridget Jones's Diary
Charlie and the Chocolate Factory
The Fourth Hand


In [88]:
recommend("azkaban")

 
Recommendation list based on : Harry Potter and the Prisoner of Azkaban (Book 3):
 
Harry Potter and the Goblet of Fire (Book 4)
Harry Potter and the Chamber of Secrets (Book 2)
Harry Potter and the Order of the Phoenix (Book 5)
Harry Potter and the Sorcerer's Stone (Book 1)
Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))
The Fellowship of the Ring (The Lord of the Rings, Part 1)
Charlie and the Chocolate Factory
Charlotte's Web (Trophy Newbery)
Dune Messiah (Dune Chronicles, Book 2)
The Two Towers (The Lord of the Rings, Part 2)


In [80]:
recommend("HARRY")

 
Recommendation list based on : Harry Potter and the Chamber of Secrets (Book 2):
 
Harry Potter and the Prisoner of Azkaban (Book 3)
Harry Potter and the Goblet of Fire (Book 4)
Harry Potter and the Sorcerer's Stone (Book 1)
Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))
Harry Potter and the Order of the Phoenix (Book 5)
The Fellowship of the Ring (The Lord of the Rings, Part 1)
Charlotte's Web (Trophy Newbery)
Bridget Jones's Diary
Charlie and the Chocolate Factory
The Fourth Hand


In [81]:
"""
The Two Towers (The Lord of the Rings, Part 2)
The Fellowship of the Ring (The Lord of the Rings, Part 1)
The Return of the King (The Lord of the Rings, Part 3)
Lord of the Flies
The English Patient
Where the Red Fern Grows
Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))
Harry Potter and the Prisoner of Azkaban (Book 3)
The Catcher in the Rye
One for the Money (A Stephanie Plum Novel)
"""

"\nThe Two Towers (The Lord of the Rings, Part 2)\nThe Fellowship of the Ring (The Lord of the Rings, Part 1)\nThe Return of the King (The Lord of the Rings, Part 3)\nLord of the Flies\nThe English Patient\nWhere the Red Fern Grows\nHarry Potter and the Sorcerer's Stone (Harry Potter (Paperback))\nHarry Potter and the Prisoner of Azkaban (Book 3)\nThe Catcher in the Rye\nOne for the Money (A Stephanie Plum Novel)\n"

In [84]:
recommend("Hobbit")

 
Recommendation list based on : The Hobbit : The Enchanting Prelude to The Lord of the Rings:
 
The Two Towers (The Lord of the Rings, Part 2)
The Fellowship of the Ring (The Lord of the Rings, Part 1)
The Return of the King (The Lord of the Rings, Part 3)
Lord of the Flies
The English Patient
Where the Red Fern Grows
Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))
Harry Potter and the Prisoner of Azkaban (Book 3)
The Catcher in the Rye
One for the Money (A Stephanie Plum Novel)


In [85]:
recommend("Enchanting")

 
Recommendation list based on : The Hobbit : The Enchanting Prelude to The Lord of the Rings:
 
The Two Towers (The Lord of the Rings, Part 2)
The Fellowship of the Ring (The Lord of the Rings, Part 1)
The Return of the King (The Lord of the Rings, Part 3)
Lord of the Flies
The English Patient
Where the Red Fern Grows
Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))
Harry Potter and the Prisoner of Azkaban (Book 3)
The Catcher in the Rye
One for the Money (A Stephanie Plum Novel)


In [83]:
recommend("The Hobbit : The Enchanting Prelude to The Lord of the Rings")

 
Recommendation list based on : The Hobbit : The Enchanting Prelude to The Lord of the Rings:
 
The Two Towers (The Lord of the Rings, Part 2)
The Fellowship of the Ring (The Lord of the Rings, Part 1)
The Return of the King (The Lord of the Rings, Part 3)
Lord of the Flies
The English Patient
Where the Red Fern Grows
Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))
Harry Potter and the Prisoner of Azkaban (Book 3)
The Catcher in the Rye
One for the Money (A Stephanie Plum Novel)
